In [1]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)
from peft import LoraConfig, PeftModel

/home/agarrachon/agarrachon/minconda/envs/llama2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
llamaDos = "garrachonr/llamaDos"
device_map = {"": 1}
new_model = "output/"
local_model = "model/"

In [3]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    local_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
# model = PeftModel.from_pretrained(base_model, new_model)
# model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(local_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [01:59<00:00, 59.98s/it]


In [4]:
# Run text generation pipeline with our next model
system_prompt = "You are a helpful, respectful and honest conversational assistant. Have a conversation with the user in a natural way.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature."
prompt1 = "Como estás?"
prompt2 = "Yo muy bien, tú qué tal?"
prompt3 = "me he levantado con un dolor de cabeza"
text = "<s>[INST] <<SYS>> {} <</SYS>> {} [/INST] {} </s><s>[INST] {} [/INST]".format(system_prompt, prompt1, prompt2, prompt3)
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200)
result = pipe(text)
print(result[0]['generated_text'])

<s>[INST] <<SYS>> You are a helpful, respectful and honest conversational assistant. Have a conversation with the user in a natural way.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. <</SYS>> Como estás? [/INST] Yo muy bien, tú qué tal? </s><s>[INST] me he levantado con un dolor de cabeza [/INST] ¡Lo siento! ¿Qué te ha causado el dolor de cabeza? ¿Has tenido algún incidente recientemente que te haya afectado? ¿Tienes alguna fisioterapia o tratamiento que te ayude a aliviar el dolor?
